# 100 Prepare input scenario database
In this notebook, we will prepare the input scenario database.

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
AR6_FILTERS = {
    'IMP_marker':[
        'SP', 'Ren'
    ],
    #'Category':'C1'
}

<IPython.core.display.Javascript object>

In [3]:
import pyam
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import dotenv
import os

from utils import calculate_netzero

import sys
sys.path.append('../scripts')
from cdr import compile_novel_cdr_estimates

<IPython.core.display.Javascript object>

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


<IPython.core.display.Javascript object>

In [4]:
dotenv.load_dotenv()

True

<IPython.core.display.Javascript object>

## Read in the input dataframe
The file should be in the `data` folder and is called: `20220314_ar6emissions_harmonized_infilled.csv`

In [5]:
df_input = pyam.IamDataFrame(
    Path(
        '../data/20220314_ar6emissions_harmonized_infilled.csv'
    )
)

pyam.core - INFO: Reading file ../data/20220314_ar6emissions_harmonized_infilled.csv


<IPython.core.display.Javascript object>

For now, we will also add in the metadata that comes along with the AR6 scenario database.

In [6]:
df_input.load_meta(
    Path(
        os.environ['AR6_META']
    )
)

pyam.core - WARNING: No meta indicators for the following scenarios:
         model           scenario
0  AIM/CGE 2.2  EN_INDCi2030_1000
1  AIM/CGE 2.2  EN_INDCi2030_500f
2  AIM/CGE 2.2  EN_INDCi2030_600f
3  AIM/CGE 2.2  EN_INDCi2030_700f
4  AIM/CGE 2.2   EN_INDCi2030_800
...
pyam.core - WARNING: Ignoring meta indicators for the following scenarios:
         model     scenario
0  AIM/CGE 2.1  CO_2Deg2020
1  AIM/CGE 2.1  CO_2Deg2030
2  AIM/CGE 2.1       CO_BAU
3  AIM/CGE 2.1       CO_GPP
4  AIM/CGE 2.1   CO_NDCplus
...


<IPython.core.display.Javascript object>

We will also need the original scenario database for the CDR variables.

In [7]:
df_ar6_csv = pd.read_csv(
    Path(
        os.environ['AR6_GLOBAL']
    )
)

<IPython.core.display.Javascript object>

In [8]:
df_ar6_csv = df_ar6_csv[df_ar6_csv.Variable.str.contains('Carbon Sequestration')]

<IPython.core.display.Javascript object>

In [9]:
df_ar6_csv.Variable.unique()

array(['Carbon Sequestration|CCS', 'Carbon Sequestration|CCS|Biomass',
       'Carbon Sequestration|CCS|Biomass|Energy|Supply',
       'Carbon Sequestration|CCS|Biomass|Energy|Supply|Electricity',
       'Carbon Sequestration|CCS|Biomass|Energy|Supply|Liquids',
       'Carbon Sequestration|CCS|Fossil',
       'Carbon Sequestration|CCS|Fossil|Energy|Demand|Industry',
       'Carbon Sequestration|CCS|Fossil|Energy|Supply',
       'Carbon Sequestration|CCS|Fossil|Energy|Supply|Electricity',
       'Carbon Sequestration|CCS|Fossil|Energy|Supply|Liquids',
       'Carbon Sequestration|Land Use',
       'Carbon Sequestration|Land Use|Afforestation',
       'Carbon Sequestration|CCS|Industrial Processes',
       'Carbon Sequestration|CCS|Biomass|Energy',
       'Carbon Sequestration|CCS|Fossil|Energy',
       'Carbon Sequestration|CCS|Fossil|Industrial Processes',
       'Carbon Sequestration|Direct Air Capture',
       'Carbon Sequestration|Enhanced Weathering',
       'Carbon Sequestration|L

<IPython.core.display.Javascript object>

In [10]:
df_ar6 = pyam.IamDataFrame(
    df_ar6_csv
)

<IPython.core.display.Javascript object>

In [11]:
df_ar6.load_meta(
    Path(
        os.environ['AR6_META']
    )
)

pyam.core - WARNING: Ignoring meta indicators for the following scenarios:
         model                scenario
0  AIM/CGE 2.0         EMF30_BCOC-EndU
1  AIM/CGE 2.0          EMF30_Baseline
2  AIM/CGE 2.0          EMF30_CH4-Only
3  AIM/CGE 2.0        EMF30_D_BCOC-Red
4  AIM/CGE 2.0  EMF30_D_CH4-ClimPolicy
...


<IPython.core.display.Javascript object>

## Data processing

Step 1: Filter out the necessary scenarios

In [12]:
df_proc = df_input.filter(
    **AR6_FILTERS
)

<IPython.core.display.Javascript object>

Step 2: Check if total CO2 emissions are reported.

In [14]:
df_proc.require_data(
    variable='AR6 climate diagnostics|Infilled|Emissions|CO2'
)

,model,scenario,variable
0,REMIND-MAgPIE 2.1-4.2,SusDev_SDP-PkBudg1000,AR6 climate diagnostics|Infilled|Emissions|CO2
1,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_ HighRE_Budg900,AR6 climate diagnostics|Infilled|Emissions|CO2


<IPython.core.display.Javascript object>

Step 3: It appears not. So let us construct this variable.

In [ ]:
df_proc.aggregate(
    variable='AR6 climate diagnostics|Infilled|Emissions|CO2',
    components=[
        'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
        'AR6 climate diagnostics|Infilled|Emissions|CO2|AFOLU'
    ],
    append=True
)

<IPython.core.display.Javascript object>

Step 4: Assign the year of net zero CO2.

In [ ]:
df_proc.set_meta(
    meta=df_proc.meta['Year of netzero CO2 emissions (Harm-Infilled) Table SPM2'],
    'netzero|CO2'
)

<IPython.core.display.Javascript object>

Step 5: Filter out for the novel CDR variables and the necessary scenarios in the AR6 scenario database.

In [ ]:
novel_cdr = compile_novel_cdr_estimates(
    df_ar6,
    AR6_FILTERS
)

{'novel_cdr_vars': ['Carbon Sequestration|Direct Air Capture', 'Carbon Sequestration|Enhanced Weathering', 'Carbon Sequestration|CCS|Biomass'], 'netzero_column': 'netzero|CO2'}
['Carbon Sequestration|CCS|Biomass', 'Carbon Sequestration|Direct Air Capture', 'Carbon Sequestration|Enhanced Weathering']


<IPython.core.display.Javascript object>

## Save out the data

In [ ]:
df_proc.to_csv(
    Path(
        '../data/100_scenarios.csv'
    )
)

<IPython.core.display.Javascript object>

In [ ]:
df_proc.export_meta(
    Path(
        '../data/100_meta.xlsx'
    )
)

<IPython.core.display.Javascript object>

In [ ]:
novel_cdr.to_csv(
    Path(
        '../data/100_novel_cdr.csv'
    )
)

<IPython.core.display.Javascript object>

Lastly, we also want to read out the scenarios list.

In [ ]:
(
    df_proc
    .meta
    .index
    .to_frame()
    .reset_index(drop=True)
    .to_csv(
        Path(
            '../data/100_scenario_names.csv'
        ),
        header=False
    )
)

<IPython.core.display.Javascript object>